In [1]:
import numpy as np
from activation_function import sigmoid, relu, sigmoid_backward, relu_backward
import matplotlib.pyplot as plt
from matplotlib import style
import os, random, cv2, pickle

style.use('ggplot')

In [4]:
sets = ['train', 'test']
DATADIR = "D:\Dataset\intel-image-classification"
CATEGORIES = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
training_data = []
testing_data = []
IMG_SIZE = 150

for s in sets:
    for category in CATEGORIES:
        path = os.path.join(DATADIR, str("seg_" + s), str("seg_" + s), category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img = os.path.join(path, img)
                img_array = cv2.imread(img)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                if s == "train":
                    training_data.append([new_array, class_num])
                elif s == "test":
                    testing_data.append([new_array, class_num])
            except Exception as e:
                print(e)

random.shuffle(training_data)
random.shuffle(testing_data)
print(len(training_data), len(testing_data))

14034 3000


In [5]:
X_train = []
y_train = []
X_test = []
y_test = []

for feature, label in training_data:
    X_train.append(feature)
    y_train.append(label)

for feature, label in testing_data:
    X_test.append(feature)
    y_test.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(14034, 150, 150, 3) (14034,) (3000, 150, 150, 3) (3000,)


In [ ]:
pickle_out_X = open('X_train.pickle', 'wb')
pickle.dump(X_train, pickle_out_X)
pickle_out_X.close()

pickle_out_y = open('y_train.pickle', 'wb')
pickle.dump(y_train, pickle_out_y)
pickle_out_y.close()

pickle_out_X = open('X_test.pickle', 'wb')
pickle.dump(X_test, pickle_out_X)
pickle_out_X.close()

pickle_out_y = open('y_test.pickle', 'wb')
pickle.dump(y_test, pickle_out_y)
pickle_out_y.close()

In [ ]:
pickle_in = open('X_train.pickle', 'rb')
X_train = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y_train.pickle', 'rb')
y_train = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('X_test.pickle', 'rb')
X_test = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y_test.pickle', 'rb')
y_test = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
# Exploring the dataset

print('Number of training examples:', X_train.shape[0])
print('Number of testing examples:', X_test.shape[0])
print('Each image is of size:', X_train[0].shape)
# X_train = X_train.reshape(X_train.shape[0], X_train[0].shape[0], X_train[0].shape[1], X_train[0].shape[2])
print(X_train)

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
        n_x - Size of input layer
        n_h - Size of hidden layer
        n_y - Size of output layer
    """
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random_randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
    
    return parameters

In [ ]:
def linear_forward(A, W, b):
    
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    if activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    return A, cache